# Seasonal Trading Plan Project
### This project seeks to figure out which stocks consistently go up over 30, 60 or 90 day periods, year after year, at least 80% of the time. It turns out there are many of them.

In this notebook, I created a process to download full historical EOD price data on each of the S&P 500 index components and analyze the historical patterns to find situations where seasonal trends can be taken advantage of over the course of a year.

For each stock, I calculated the following statistics:

Holding Period:  The number of days held in each rolling period - 30, 60 and 90 days each.

% Up Rows:  The percentage of rolling periods where, each year, the stock went up at least 80% of the time.

Avg Up Return:  The average return for each rolling period where the return was positive.

Avg Up StDev: The standard deviation of return for each rolling period where the return was positive. This means that the expected return should be within this +/- range from the average 67% of the time.

% Downside:  The expected average return minus the standard deviation. This means that you should at least exceed this return 67% of the time. This could provide guidance for setting stop-loss levels.

Least Pain Pt:  The time interval that offers the best case 67% downside within the sweet spot period. This is used to pinpoint the lowest risk time frame.

Best Begin Date:  The recommended entry date for best risk/reward scenario.

Best End Date:  The recommended exit date for best risk/reward scenario.

Max Consec 80%+:  The longest consecutive number of rolling periods above 80% up years. You want to see more that a few consecutive up years over the threshold to provide assurance that the trend is robust.

Total Years:  The number of years of historical data available for each stock. I filtered for stocks with 10+ years of history.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr

In [2]:
# This is the path to where I store this notebook and downloaded files. You need to change this to your convenient
# spot on your own hard drive.

my_path = '/Users/bnsheehy/Documents/Investments/Seasonal_Analyses/Python'
threshold = 0.80

In [3]:
file_sp500_tickers = my_path + "/SP500_TickerList.csv"

In [4]:
# Upload a list of the S&P 500 components downloaded from Yahoo.

df_sp500_tickers = pd.read_csv (file_sp500_tickers)
df_sp500_tickers.head()

,Symbol,Company Name,GICS Sector,GICS Sub Industry
0,A,Agilent Technologies Inc,Health Care,Health Care Equipment
1,AAL,American Airlines Group,Industrials,Airlines
2,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail
3,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals"
4,ABBV,Rickers,Health Care,Pharmaceuticals


In [5]:
# This module loops through the S&P 500 tickers, downloads the data from Yahoo and creates a separate CSV 
# file of historical data for each ticker (e.g. AAPL.csv).
# Skip this routine if you already have the CSV files available.

for index, ticker in df_sp500_tickers.iterrows():
    my_ticker = ticker['Symbol']
    
    yf_ticker = yf.Ticker(my_ticker)
    data = yf_ticker.history(period="max")
    df = pd.DataFrame(data)
    df.reset_index(level=0, inplace=True)
    df['Symbol'] = my_ticker
    df = df[['Symbol','Date','Close']]
    df.to_csv(path_or_buf = my_path + "/data/" + my_ticker +".csv", index=False)
    

In [5]:
# Creates the dataframe container for the stats data.

df_tradelist = pd.DataFrame(index=[], columns=['my_ticker', 'hold_per', 'pct_uprows', 'max_up_return', 'min_up_return', 'avg_up_return', 'stdev_up_return', 'pct_downside', 'least_pain_pt', 'total_years', 'max_consec_beat', 'best_buy_date', 'best_sell_date'])

df_tradelist.head()

,my_ticker,hold_per,pct_uprows,max_up_return,min_up_return,avg_up_return,stdev_up_return,pct_downside,least_pain_pt,total_years,max_consec_beat,best_buy_date,best_sell_date


In [6]:
# This module grabs each ticker file, transforms it and calculates the statistics needed for a 90 day holding period.

def calc_3month_returns():
    
    global df_tradelist
    global threshold
    hold_per="3 Mos"
    
    # Convert prices to 3 month returns based on 20 trading days per month.
    dfr = df.pct_change(periods=60)
    dfr.reset_index(level=0, inplace=True)
    dfr.rename(columns={'Close':'Returns'}, inplace=True)
    dfr = dfr.round(4)
    
    # Separate out the date column into separate month, year and day values.
    dfr['Month'] = pd.DatetimeIndex(dfr['Date']).month
    dfr['Day'] = pd.DatetimeIndex(dfr['Date']).day
    dfr['Year'] = pd.DatetimeIndex(dfr['Date']).year
    dfr['M-D'] = dfr['Month'].astype(str)+'-'+dfr['Day'].astype(str)
    pd.set_option('display.max_rows', len(dfr))

    # Pivot the table to show years across the top and Month-Day values in the first column on the left.
    dfr_pivot = dfr.pivot(index='M-D', columns='Year', values='Returns')
    dfr_pivot.reset_index(level=0, inplace=True)
    dfr_pivot = pd.DataFrame(dfr_pivot)
    dfr_pivot.columns.name="Index"

    # The pivot operation created empty cells for weekends and holiday, so I filled them with EOD values from
    # the previous trading day.
    dfr_pivot.fillna(method='ffill', inplace=True)

    # Add additional calculated columns to facilitate statistic calculations for each stock.
    dfr_pivot['YearCount'] = dfr_pivot.count(axis=1, numeric_only=True)
    dfr_pivot['UpCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] > 0].count(axis=1)-1
    dfr_pivot['DownCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] < 0].count(axis=1)
    dfr_pivot['PctUp'] = dfr_pivot['UpCount']/dfr_pivot['YearCount']
    dfr_pivot['PctDown'] = dfr_pivot['DownCount']/dfr_pivot['YearCount']
    dfr_pivot['AvgReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-5].mean(axis=1)
    dfr_pivot['StDevReturns'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-6].std(axis=1)
    dfr_pivot['67PctDownside'] = dfr_pivot['AvgReturn']-dfr_pivot['StDevReturns']
    dfr_pivot['MaxReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-8].max(axis=1)
    dfr_pivot['MinReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-9].min(axis=1)

    # Add a fictional date column in Python date/time format so the table can be sorted by date. Then sort by Date.
    dfr_pivot['Date'] = '2000-' + dfr_pivot['M-D'].astype(str)
    dfr_pivot['Date'] = pd.to_datetime(dfr_pivot['Date'], infer_datetime_format=True)
    dfr_pivot.sort_values(by='Date',ascending=True, inplace=True)
    
    # Reset the index and round the float values to 4 decimals.
    dfr_pivot.reset_index(inplace=True)
    dfr_pivot = dfr_pivot.round(4)
    
    # Export the final pivot table to CSV for further research later.
    dfr_pivot.to_csv(path_or_buf = my_path + "/data/" + my_ticker + "_dfr_pivot_3mo.csv", index=False)

    # Calculate the trading statistics for the rolling holding periods for the stock.
    pct_uprows = (dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'PctUp'].count() / dfr_pivot.loc[:, 'PctUp'].count()).astype(float).round(4)
    max_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MaxReturn'].max()
    min_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MinReturn'].min()
    avg_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'AvgReturn'].mean()
    avg_up_return = np.float64(avg_up_return).round(4)
    stdev_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'StDevReturns'].mean()
    stdev_up_return = np.float64(stdev_up_return).round(4)
    pct_downside = (avg_up_return - stdev_up_return)
    pct_downside = np.float64(pct_downside).round(4)
    least_pain_pt = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, '67PctDownside'].max()
    total_years = dfr_pivot['YearCount'].max()
    
    n_consec = 0
    max_n_consec = 0

    for x in dfr_pivot['PctUp']:
        if (x > threshold):
            n_consec += 1
        else: # check for new max, then start again from 1
            max_n_consec = max(n_consec, max_n_consec)
            n_consec = 1

    max_consec_beat = max_n_consec

    try:
        best_sell_date = dfr_pivot.loc[dfr_pivot['67PctDownside'] == least_pain_pt, 'M-D'].iloc[0]
    except:
        best_sell_date = "nan"

    try:
        row = dfr_pivot.loc[dfr_pivot['M-D'] == best_sell_date, 'M-D'].index[0]-90
        col = dfr_pivot.columns.get_loc('M-D')
        best_buy_date = dfr_pivot.iloc[row,col]
    except:
        best_buy_date = "nan"

    # If the pct_uprows and history conditions are met, then create the array of stat values and append 
    # it to the recommended trade list.
    if pct_uprows > 0.1:
        if total_years > 9:
            statsdata = np.array([my_ticker, hold_per, pct_uprows, max_up_return, min_up_return, avg_up_return, stdev_up_return, pct_downside, least_pain_pt, total_years, max_consec_beat, best_buy_date, best_sell_date])
            df_tradelist = df_tradelist.append(dict(zip(df_tradelist.columns, statsdata)), ignore_index=True)
            

In [7]:
# This module grabs each ticker file, transforms it and calculates the statistics needed for a 60 day holding period.

def calc_2month_returns():
    
    global df_tradelist
    global threshold
    hold_per="2 Mos"

    # Convert prices to 3 month returns based on 20 trading days per month.
    dfr = df.pct_change(periods=40)
    dfr.reset_index(level=0, inplace=True)
    dfr.rename(columns={'Close':'Returns'}, inplace=True)
    dfr = dfr.round(4)
    
    # Separate out the date column into separate month, year and day values.
    dfr['Month'] = pd.DatetimeIndex(dfr['Date']).month
    dfr['Day'] = pd.DatetimeIndex(dfr['Date']).day
    dfr['Year'] = pd.DatetimeIndex(dfr['Date']).year
    dfr['M-D'] = dfr['Month'].astype(str)+'-'+dfr['Day'].astype(str)
    pd.set_option('display.max_rows', len(dfr))

    # Pivot the table to show years across the top and Month-Day values in the first column on the left.
    dfr_pivot = dfr.pivot(index='M-D', columns='Year', values='Returns')
    dfr_pivot.reset_index(level=0, inplace=True)
    dfr_pivot = pd.DataFrame(dfr_pivot)
    dfr_pivot.columns.name="Index"

    # The pivot operation created empty cells for weekends and holiday, so I filled them with EOD values from
    # the previous trading day.
    dfr_pivot.fillna(method='ffill', inplace=True)

    # Add additional calculated columns to facilitate statistic calculations for each stock.
    dfr_pivot['YearCount'] = dfr_pivot.count(axis=1, numeric_only=True)
    dfr_pivot['UpCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] > 0].count(axis=1)-1
    dfr_pivot['DownCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] < 0].count(axis=1)
    dfr_pivot['PctUp'] = dfr_pivot['UpCount']/dfr_pivot['YearCount']
    dfr_pivot['PctDown'] = dfr_pivot['DownCount']/dfr_pivot['YearCount']
    dfr_pivot['AvgReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-5].mean(axis=1)
    dfr_pivot['StDevReturns'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-6].std(axis=1)
    dfr_pivot['67PctDownside'] = dfr_pivot['AvgReturn']-dfr_pivot['StDevReturns']
    dfr_pivot['MaxReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-8].max(axis=1)
    dfr_pivot['MinReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-9].min(axis=1)

    # Add a fictional date column in Python date/time format so the table can be sorted by date. Then sort by Date.
    dfr_pivot['Date'] = '2000-' + dfr_pivot['M-D'].astype(str)
    dfr_pivot['Date'] = pd.to_datetime(dfr_pivot['Date'], infer_datetime_format=True)
    dfr_pivot.sort_values(by='Date',ascending=True, inplace=True)
    
    # Reset the index and round the float values to 4 decimals.
    dfr_pivot.reset_index(inplace=True)
    dfr_pivot = dfr_pivot.round(4)
    
    # Export the final pivot table to CSV for further research later.
    dfr_pivot.to_csv(path_or_buf = my_path + "/data/" + my_ticker + "_dfr_pivot_2mo.csv", index=False)

    # Calculate the trading statistics for the rolling holding periods for the stock.
    pct_uprows = (dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'PctUp'].count() / dfr_pivot.loc[:, 'PctUp'].count()).astype(float).round(4)
    max_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MaxReturn'].max()
    min_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MinReturn'].min()
    avg_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'AvgReturn'].mean()
    avg_up_return = np.float64(avg_up_return).round(4)
    stdev_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'StDevReturns'].mean()
    stdev_up_return = np.float64(stdev_up_return).round(4)
    pct_downside = (avg_up_return - stdev_up_return)
    pct_downside = np.float64(pct_downside).round(4)
    least_pain_pt = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, '67PctDownside'].max()
    total_years = dfr_pivot['YearCount'].max()
    
    n_consec = 0
    max_n_consec = 0

    for x in dfr_pivot['PctUp']:
        if (x > threshold):
            n_consec += 1
        else: # check for new max, then start again from 1
            max_n_consec = max(n_consec, max_n_consec)
            n_consec = 1

    max_consec_beat = max_n_consec

    try:
        best_sell_date = dfr_pivot.loc[dfr_pivot['67PctDownside'] == least_pain_pt, 'M-D'].iloc[0]
    except:
        best_sell_date = "nan"

    try:
        row = dfr_pivot.loc[dfr_pivot['M-D'] == best_sell_date, 'M-D'].index[0]-60
        col = dfr_pivot.columns.get_loc('M-D')
        best_buy_date = dfr_pivot.iloc[row,col]
    except:
        best_buy_date = "nan"

    # If the pct_uprows and history conditions are met, then create the array of stat values and append 
    # it to the recommended trade list.
    if pct_uprows > 0.1:
        if total_years > 9:
            statsdata = np.array([my_ticker, hold_per, pct_uprows, max_up_return, min_up_return, avg_up_return, stdev_up_return, pct_downside, least_pain_pt, total_years, max_consec_beat, best_buy_date, best_sell_date])
            df_tradelist = df_tradelist.append(dict(zip(df_tradelist.columns, statsdata)), ignore_index=True)
            

In [8]:
# This module grabs each ticker file, transforms it and calculates the statistics needed for a 30 day holding period.

def calc_1month_returns():
    
    global df_tradelist
    global threshold
    hold_per="1 Mo"

    # Convert prices to 3 month returns based on 20 trading days per month.
    dfr = df.pct_change(periods=20)
    dfr.reset_index(level=0, inplace=True)
    dfr.rename(columns={'Close':'Returns'}, inplace=True)
    dfr = dfr.round(4)
    
    # Separate out the date column into separate month, year and day values.
    dfr['Month'] = pd.DatetimeIndex(dfr['Date']).month
    dfr['Day'] = pd.DatetimeIndex(dfr['Date']).day
    dfr['Year'] = pd.DatetimeIndex(dfr['Date']).year
    dfr['M-D'] = dfr['Month'].astype(str)+'-'+dfr['Day'].astype(str)

    # Pivot the table to show years across the top and Month-Day values in the first column on the left.
    dfr_pivot = dfr.pivot(index='M-D', columns='Year', values='Returns')
    dfr_pivot.reset_index(level=0, inplace=True)
    dfr_pivot = pd.DataFrame(dfr_pivot)
    dfr_pivot.columns.name="Index"

    # The pivot operation created empty cells for weekends and holiday, so I filled them with EOD values from
    # the previous trading day.
    dfr_pivot.fillna(method='ffill', inplace=True)

    # Add additional calculated columns to facilitate statistic calculations for each stock.
    dfr_pivot['YearCount'] = dfr_pivot.count(axis=1, numeric_only=True)
    dfr_pivot['UpCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] > 0].count(axis=1)-1
    dfr_pivot['DownCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] < 0].count(axis=1)
    dfr_pivot['PctUp'] = dfr_pivot['UpCount']/dfr_pivot['YearCount']
    dfr_pivot['PctDown'] = dfr_pivot['DownCount']/dfr_pivot['YearCount']
    dfr_pivot['AvgReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-5].mean(axis=1)
    dfr_pivot['StDevReturns'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-6].std(axis=1)
    dfr_pivot['67PctDownside'] = dfr_pivot['AvgReturn']-dfr_pivot['StDevReturns']
    dfr_pivot['MaxReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-8].max(axis=1)
    dfr_pivot['MinReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-9].min(axis=1)

    # Add a fictional date column in Python date/time format so the table can be sorted by date. Then sort by Date.
    dfr_pivot['Date'] = '2000-' + dfr_pivot['M-D'].astype(str)
    dfr_pivot['Date'] = pd.to_datetime(dfr_pivot['Date'], infer_datetime_format=True)
    dfr_pivot.sort_values(by='Date',ascending=True, inplace=True)
    
    # Reset the index and round the float values to 4 decimals.
    dfr_pivot.reset_index(inplace=True)
    dfr_pivot = dfr_pivot.round(4)
    
    # Export the final pivot table to CSV for further research later.
    dfr_pivot.to_csv(path_or_buf = my_path + "/data/" + my_ticker + "_dfr_pivot_1mo.csv", index=False)

    # Calculate the trading statistics for the rolling holding periods for the stock.
    pct_uprows = (dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'PctUp'].count() / dfr_pivot.loc[:, 'PctUp'].count()).astype(float).round(4)
    max_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MaxReturn'].max()
    min_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MinReturn'].min()
    avg_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'AvgReturn'].mean()
    avg_up_return = np.float64(avg_up_return).round(4)
    stdev_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'StDevReturns'].mean()
    stdev_up_return = np.float64(stdev_up_return).round(4)
    pct_downside = (avg_up_return - stdev_up_return)
    pct_downside = np.float64(pct_downside).round(4)
    least_pain_pt = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, '67PctDownside'].max()
    total_years = dfr_pivot['YearCount'].max()

    n_consec = 0
    max_n_consec = 0

    for x in dfr_pivot['PctUp']:
        if (x > threshold):
            n_consec += 1
        else: # check for new max, then start again from 1
            max_n_consec = max(n_consec, max_n_consec)
            n_consec = 1

    max_consec_beat = max_n_consec

    try:
        best_sell_date = dfr_pivot.loc[dfr_pivot['67PctDownside'] == least_pain_pt, 'M-D'].iloc[0]
    except:
        best_sell_date = "nan"

    try:
        row = dfr_pivot.loc[dfr_pivot['M-D'] == best_sell_date, 'M-D'].index[0]-30
        col = dfr_pivot.columns.get_loc('M-D')
        best_buy_date = dfr_pivot.iloc[row,col]
    except:
        best_buy_date = "nan"

    # If the pct_uprows and history conditions are met, then create the array of stat values and append 
    # it to the recommended trade list.
    if pct_uprows > 0.1:
        if total_years > 9:
            statsdata = np.array([my_ticker, hold_per, pct_uprows, max_up_return, min_up_return, avg_up_return, stdev_up_return, pct_downside, least_pain_pt, total_years, max_consec_beat, best_buy_date, best_sell_date])
            df_tradelist = df_tradelist.append(dict(zip(df_tradelist.columns, statsdata)), ignore_index=True)
            

In [9]:

# Read CSV files by ticker, transform and extract stats from each one.

for index, ticker in df_sp500_tickers.iterrows():
    my_ticker = ticker['Symbol']

    df = pd.read_csv (my_path + "/data/" + my_ticker + ".csv")
    df.set_index('Date', inplace=True)
    df = df['Close']
    df = pd.DataFrame(df, columns=['Close'])
    
    calc_1month_returns()
    
    calc_2month_returns()
    
    calc_3month_returns()




In [10]:
# Preview the end of the trade list to see how many rows and make sure the data is showing up where it is supposed to.

df_tradelist.tail(10)

,my_ticker,hold_per,pct_uprows,max_up_return,min_up_return,avg_up_return,stdev_up_return,pct_downside,least_pain_pt,total_years,max_consec_beat,best_buy_date,best_sell_date
88,V,2 Mos,0.1625,0.3951,-0.3084,0.0404,0.0838,-0.0434,0.0063,13,10,5-22,7-22
89,V,3 Mos,0.3444,0.4428,-0.3825,0.0541,0.1091,-0.055,0.0063,13,16,10-6,1-6
90,VRSK,1 Mo,0.1791,0.1499,-0.1418,0.0197,0.045,-0.0253,0.0078,12,12,6-6,7-7
91,VRSK,2 Mos,0.2479,0.1863,-0.1211,0.0395,0.0646,-0.0251,0.0107,12,22,5-26,7-26
92,VRSK,3 Mos,0.3691,0.2642,-0.147,0.0405,0.0689,-0.0284,0.0264,11,33,4-11,7-11
93,WCG,1 Mo,0.1433,1.0482,-0.6091,0.0335,0.132,-0.0985,0.0095,17,7,5-3,6-2
94,XYL,1 Mo,0.1267,0.1956,-0.19,0.0193,0.0537,-0.0344,0.0218,10,11,8-22,9-21
95,XYL,2 Mos,0.1102,0.2747,-0.1352,0.0295,0.072,-0.0425,0.0133,10,10,1-5,3-5
96,YUM,2 Mos,0.1543,0.5174,-0.2408,0.0309,0.0939,-0.063,0.0225,24,25,2-25,4-25
97,YUM,3 Mos,0.1377,0.5091,-0.2379,0.0345,0.1169,-0.0824,0.0246,24,39,2-6,5-6


In [11]:
# Make a copy and convert the trade list to a Pandas dataframe.
df_tradelist_copy = df_tradelist.copy()
df_tradelist = pd.DataFrame(df_tradelist)

In [18]:
# Clean it up by removing rows with NaN's and infinity values.
df_tradelist.replace("inf", np.nan, inplace=True)
df_tradelist.dropna(inplace=True)
df_tradelist = df_tradelist[~df_tradelist.max_up_return.str.contains("nan")]
df_tradelist.sort_values(by=['pct_uprows'], ascending=False)
df_tradelist.drop_duplicates(subset ="my_ticker", keep = 'first', inplace = True) 

In [21]:
df_tradelist.head()
#df_tradelist.shape

,my_ticker,hold_per,pct_uprows,max_up_return,min_up_return,avg_up_return,stdev_up_return,pct_downside,least_pain_pt,total_years,max_consec_beat,best_buy_date,best_sell_date
0,A,3 Mos,0.1019,0.8236,-0.4991,0.0403,0.2078,-0.1675,0.0013,21,13,11-1,2-1
1,AIZ,3 Mos,0.1047,0.4205,-0.7749,0.0426,0.1234,-0.0808,-0.0136,17,19,3-21,6-19
2,AME,3 Mos,0.1267,0.6667,-0.3333,0.0602,0.1391,-0.0789,-0.0122,37,12,1-24,4-23
3,AMP,1 Mo,0.1129,0.609,-0.4184,0.0201,0.0907,-0.0706,-0.0003,16,12,8-22,9-21
6,AVGO,2 Mos,0.157,0.4867,-0.1815,0.0467,0.0964,-0.0497,0.02,12,17,1-28,3-28


In [22]:
# Export the trade list to CSV files for execution and/or further research if desired.
df_tradelist.to_csv(path_or_buf = my_path + "/df_tradelist.csv", index=False)